# TensorFlow Serving

Fornisce un'integrazione pronta all'uso per poter "servire" modelli di Machine Learning.
 
E' uno strumento studiato per gli ambienti di produzione.

https://www.tensorflow.org/tfx/guide/serving

## Docker example

### Installazione

_docker pull tensorflow/serving_

## Costruzione del modello

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import json
import time

from random import randint

### Preparazione fake data e del modello

In [2]:
X1 = [randint(0,100) for i in range(1000)]
X2 = [randint(-50,50) for i in range(1000)]

In [3]:
df = pd.DataFrame({"x1" : X1, "x2" : X2})

In [4]:
def get_y(x):
    
    y = 2*x[0] + x[1]/2
    
    return y

In [5]:
df

,x1,x2
0,40,5
1,4,-39
2,99,31
3,73,50
4,2,-46
...,...,...
995,34,4
996,29,-28
997,16,-39
998,82,18


In [6]:
df['y'] = df.apply(get_y, axis=1)

In [7]:
df.head()

,x1,x2,y
0,40,5,82.5
1,4,-39,-11.5
2,99,31,213.5
3,73,50,171.0
4,2,-46,-19.0


In [8]:
X = df[['x1', 'x2']].values
y = df.y.values

In [21]:
with tf.device('/cpu:0'):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(100, input_shape=(2,)))
    model.add(keras.layers.Dense(1))

    model.compile(loss='mae', optimizer='adam', metrics='mae')
    
    model.fit(X,y, epochs=100, verbose=0)

In [10]:
with tf.device('/cpu:0'):
    y_hat = model.predict(X[0].reshape(-1,2).tolist())

In [11]:
y_hat

array([[82.6986]], dtype=float32)

### Salvataggio del modello

In [12]:
# Nome del modello
name = 'example'

# Versione del modello (TensorFlow Serving gestisce automaticamente l'ultima versione)
version = int(time.time())

# Salvataggio
model.save(filepath=f'models/{name}/{version}', save_format='pb', overwrite=True)

INFO:tensorflow:Assets written to: models/example/1676766440/assets


2023-02-19 01:27:21.002377: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


#### Comando di inspect del modello

**saved_model_cli show --dir models/example/001 --tag_set serve --signature_def serving_default**

In [13]:
print("""The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_2_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: serving_default_dense_2_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict""")

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_2_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: serving_default_dense_2_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


### Avvio del modello attraverso un'istanza docker

**path dei modelli**

MODEL_DIR="$(pwd)/models"

All'interno di questa cartella ci sarà dunque ogni singolo modello in una cartella (example in questo caso), all'interno di ogni cartella ci sarà una sottocartella numerica che conterrà la versione del modello.

**avvio dell'istanza docker per il modello _example_**

docker run -t --rm -p 8501:8501 -v "$MODEL_DIR/example:/models/example" -e MODEL_NAME=example tensorflow/serving &

**esempi di interrogazione attraverso curl**

curl -d "{\"instances\": [[10]]}" -X POST http://localhost:8501/v1/models/example:predict

{
    "predictions": [[24.9380608]
    ]
}

curl -d "{\"instances\": [[10], [10], [20]]}" -X POST http://localhost:8501/v1/models/example:predict

### Interrogazione API attraverso la libreria "requests"

In [3]:
import requests
import numpy as np

In [4]:
data = dict()

In [5]:
x1 = 1
x2 = 2

In [6]:
np.array([x1,x2])

array([1, 2])

In [7]:
data = {"instances" : np.array([x1,x2]).reshape(-1,2).tolist()}
data

{'instances': [[1, 2]]}

In [8]:
url = "http://localhost:8501/v1/models/example:predict"
# url = "http://localhost:8501/v1/models/example:classify"

In [9]:
response = requests.post(url, json=data)

In [10]:
response.status_code

200

In [11]:
response.json()

{'predictions': [[2.99026871]]}